<h1>ENB33 - Revisiting FFNs with K-fold cross validation</h1>
<h2> different no. of neurons per layer </h2>

In [1]:
import tensorflow as tf
import keras
from keras import models
from keras import layers

import numpy as np
import os
import csv

import pickle
import time as tm

import matplotlib.pyplot as plt
import pre_process as pp

Using TensorFlow backend.


<h2> Loading the datasets</h2>

In [2]:
with open('train_data.txt', 'rb') as fp:
    train_data = pickle.load(fp)
    
with open('train_labels.txt', 'rb') as fp:
    train_labels = pickle.load(fp)
    
with open('test_data.txt', 'rb') as fp:
    test_data = pickle.load(fp)
    
with open('test_labels.txt', 'rb') as fp:
    test_labels = pickle.load(fp)

<h2>k-fold params</h2>

In [3]:
k = 4
num_val_samples = len(train_data)
print(num_val_samples)
val_split_size = num_val_samples/k
val_split_size = int(val_split_size)
print(val_split_size)
type(val_split_size)

72
18


int

<h2>Experiment1: a single layer</h2>

In [4]:
net1 = models.Sequential()
net1.add(layers.Dense((480), input_shape = (480,6)))
net1.add(layers.Flatten())
net1.add(layers.Dense(360,activation='relu'))
net1.add(layers.Dense(240,activation='relu'))
net1.add(layers.Dense(240,activation='relu'))
net1.add(layers.Dense(7, activation = 'softmax'))
net1.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 480, 480)          3360      
_________________________________________________________________
flatten_1 (Flatten)          (None, 230400)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 360)               82944360  
_________________________________________________________________
dense_3 (Dense)              (None, 240)               86640     
_________________________________________________________________
dense_4 (Dense)              (None, 240)               57840     
_________________________________________________________________
dense_5 (Dense)              (None, 7)                 1687      
Total params: 83,093,887
Trainable para

In [5]:
net1.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
net1.save_weights('net1.h5')  # each split will start with same weights so can directly compare results

In [6]:

histories1 = []
results_list1 = []
index = 0
fold = val_split_size



start = tm.time()

for i in range (k):
    print(i)
    val_fold = train_data[index:fold]
    train_fold = np.delete(train_data, np.s_[index:fold],0)

    val_tgt = train_labels[index:fold]
    train_tgt = np.delete(train_labels, np.s_[index:fold])
    
    
    net1.load_weights('net1.h5')
    history = net1.fit(train_fold, train_tgt, epochs = 25, validation_data = (val_fold, val_tgt))

    histories1.append(history)
    
    
    results = net1.evaluate(test_data, test_labels)
    results_list1.append(results)
    print(results)
    
    index = index + val_split_size
    fold = fold + val_split_size
    
finish = tm.time()
duration1 = finish - start

0

Train on 54 samples, validate on 18 samples
Epoch 1/25
54/54 [==============================] - 4s 71ms/step - loss: 5.6269 - accuracy: 0.1111 - val_loss: 12.6446 - val_accuracy: 0.1667
Epoch 2/25
54/54 [==============================] - 2s 40ms/step - loss: 8.9221 - accuracy: 0.1481 - val_loss: 5.2251 - val_accuracy: 0.1667
Epoch 3/25
54/54 [==============================] - 2s 39ms/step - loss: 3.9891 - accuracy: 0.2037 - val_loss: 4.2614 - val_accuracy: 0.0556
Epoch 4/25
54/54 [==============================] - 2s 41ms/step - loss: 3.6198 - accuracy: 0.1111 - val_loss: 2.7073 - val_accuracy: 0.3333
Epoch 5/25
54/54 [==============================] - 2s 39ms/step - loss: 2.4179 - accuracy: 0.3519 - val_loss: 2.0056 - val_accuracy: 0.5000
Epoch 6/25
54/54 [==============================] - 2s 39ms/step - loss: 1.7947 - accuracy: 0.3704 - val_loss: 1.3741 - val_accuracy: 0.4444
Epoch 7/25
54/54 [==============================] - 2s 40ms/step - loss: 1.2881 - accuracy: 0.4074 - val_l

Epoch 5/25
54/54 [==============================] - 2s 40ms/step - loss: 3.6365 - accuracy: 0.2963 - val_loss: 1.9475 - val_accuracy: 0.2778
Epoch 6/25
54/54 [==============================] - 2s 40ms/step - loss: 2.0907 - accuracy: 0.2037 - val_loss: 3.6379 - val_accuracy: 0.2222
Epoch 7/25
54/54 [==============================] - 2s 40ms/step - loss: 2.0383 - accuracy: 0.5000 - val_loss: 3.1637 - val_accuracy: 0.3333
Epoch 8/25
54/54 [==============================] - 2s 40ms/step - loss: 2.0850 - accuracy: 0.3889 - val_loss: 1.7381 - val_accuracy: 0.2222
Epoch 9/25
54/54 [==============================] - 2s 40ms/step - loss: 1.1144 - accuracy: 0.5556 - val_loss: 1.4697 - val_accuracy: 0.3889
Epoch 10/25
54/54 [==============================] - 2s 40ms/step - loss: 1.1983 - accuracy: 0.3704 - val_loss: 1.0359 - val_accuracy: 0.4444
Epoch 11/25
54/54 [==============================] - 2s 40ms/step - loss: 0.7713 - accuracy: 0.7593 - val_loss: 1.0012 - val_accuracy: 0.4444
Epoch 12/25

In [7]:
avg = np.mean(results_list1, axis=0)
std_dev = np.std(results_list1, axis=0)
print('Average loss and accuracy scores were: ', avg)
print('Standard deviation for loss & accuracy: ', std_dev)
print('Duration in seconds: ', int(duration1))

Average loss and accuracy scores were:  [0.49632676 0.80000001]
Standard deviation for loss & accuracy:  [0.04950696 0.06666666]
Duration in seconds:  221


<h2> Expriment2:</h2>

In [8]:
net2 = models.Sequential()
net2.add(layers.Dense((480), input_shape = (480,6)))
net2.add(layers.Flatten())
net2.add(layers.Dense(360,activation='relu'))
net2.add(layers.Dense(240,activation='relu'))
net2.add(layers.Dense(120,activation='relu'))
net2.add(layers.Dense(7, activation = 'softmax'))
net2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 480, 480)          3360      
_________________________________________________________________
flatten_2 (Flatten)          (None, 230400)            0         
_________________________________________________________________
dense_7 (Dense)              (None, 360)               82944360  
_________________________________________________________________
dense_8 (Dense)              (None, 240)               86640     
_________________________________________________________________
dense_9 (Dense)              (None, 120)               28920     
_________________________________________________________________
dense_10 (Dense)             (None, 7)                 847       
Total params: 83,064,127
Trainable params: 83,064,127
Non-trainable params: 0
__________________________________________

In [9]:
net2.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
net2.save_weights('net2.h5')  # each split will start with same weights so can directly compare results

histories2 = []
results_list2 = []
index = 0
fold = val_split_size



start = tm.time()

for i in range (k):
    print(i)
    val_fold = train_data[index:fold]
    train_fold = np.delete(train_data, np.s_[index:fold],0)

    val_tgt = train_labels[index:fold]
    train_tgt = np.delete(train_labels, np.s_[index:fold])
    
    
    net2.load_weights('net2.h5')
    history = net2.fit(train_fold, train_tgt, epochs = 25, validation_data = (val_fold, val_tgt))

    histories2.append(history)
    
    
    results = net2.evaluate(test_data, test_labels)
    results_list2.append(results)
    print(results)
    
    index = index + val_split_size
    fold = fold + val_split_size

finish = tm.time()
duration2 = finish - start

0
Train on 54 samples, validate on 18 samples
Epoch 1/25
54/54 [==============================] - 2s 44ms/step - loss: 6.8845 - accuracy: 0.1667 - val_loss: 4.7517 - val_accuracy: 0.0556
Epoch 2/25
54/54 [==============================] - 2s 40ms/step - loss: 4.9962 - accuracy: 0.0926 - val_loss: 5.6310 - val_accuracy: 0.0000e+00
Epoch 3/25
54/54 [==============================] - 2s 40ms/step - loss: 6.9927 - accuracy: 0.0741 - val_loss: 5.3046 - val_accuracy: 0.1667
Epoch 4/25
54/54 [==============================] - 2s 40ms/step - loss: 6.4471 - accuracy: 0.2222 - val_loss: 2.9543 - val_accuracy: 0.3889
Epoch 5/25
54/54 [==============================] - 2s 40ms/step - loss: 3.8382 - accuracy: 0.4074 - val_loss: 2.5606 - val_accuracy: 0.1111
Epoch 6/25
54/54 [==============================] - 2s 40ms/step - loss: 2.1321 - accuracy: 0.2778 - val_loss: 2.1772 - val_accuracy: 0.2222
Epoch 7/25
54/54 [==============================] - 2s 40ms/step - loss: 1.5431 - accuracy: 0.4259 - val

54/54 [==============================] - 2s 40ms/step - loss: 1.6235 - accuracy: 0.3519 - val_loss: 2.4209 - val_accuracy: 0.2222
Epoch 7/25
54/54 [==============================] - 2s 39ms/step - loss: 1.8941 - accuracy: 0.5926 - val_loss: 1.4035 - val_accuracy: 0.6667
Epoch 8/25
54/54 [==============================] - 2s 40ms/step - loss: 1.2869 - accuracy: 0.5556 - val_loss: 2.9041 - val_accuracy: 0.2222
Epoch 9/25
54/54 [==============================] - 2s 39ms/step - loss: 2.0567 - accuracy: 0.5000 - val_loss: 3.3686 - val_accuracy: 0.2222
Epoch 10/25
54/54 [==============================] - 2s 40ms/step - loss: 1.8869 - accuracy: 0.5926 - val_loss: 2.0327 - val_accuracy: 0.4444
Epoch 11/25
54/54 [==============================] - 2s 40ms/step - loss: 1.2434 - accuracy: 0.5185 - val_loss: 1.7616 - val_accuracy: 0.5000
Epoch 12/25
54/54 [==============================] - 2s 40ms/step - loss: 0.9561 - accuracy: 0.6111 - val_loss: 0.9540 - val_accuracy: 0.4444
Epoch 13/25
54/54 [==

In [10]:
avg = np.mean(results_list2, axis=0)
std_dev = np.std(results_list2, axis=0)
print('Average loss and accuracy scores were: ', avg)
print('Standard deviation for loss & accuracy: ', std_dev)
print('Duration in seconds: ', int(duration2))

Average loss and accuracy scores were:  [0.54250474 0.75000001]
Standard deviation for loss & accuracy:  [0.05355679 0.05527708]
Duration in seconds:  218


<h2> Expriment3:</h2>

In [11]:
net3 = models.Sequential()
net3.add(layers.Dense((480), input_shape = (480,6)))
net3.add(layers.Flatten())
net3.add(layers.Dense(240,activation='relu'))
net3.add(layers.Dense(240,activation='relu'))
net3.add(layers.Dense(120,activation='relu'))
net3.add(layers.Dense(7, activation = 'softmax'))
net3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 480, 480)          3360      
_________________________________________________________________
flatten_3 (Flatten)          (None, 230400)            0         
_________________________________________________________________
dense_12 (Dense)             (None, 240)               55296240  
_________________________________________________________________
dense_13 (Dense)             (None, 240)               57840     
_________________________________________________________________
dense_14 (Dense)             (None, 120)               28920     
_________________________________________________________________
dense_15 (Dense)             (None, 7)                 847       
Total params: 55,387,207
Trainable params: 55,387,207
Non-trainable params: 0
__________________________________________

In [12]:
net3.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
net3.save_weights('net3.h5')  # each split will start with same weights so can directly compare results

histories3 = []
results_list3 = []
index = 0
fold = val_split_size



start = tm.time()

for i in range (k):
    print(i)
    val_fold = train_data[index:fold]
    train_fold = np.delete(train_data, np.s_[index:fold],0)

    val_tgt = train_labels[index:fold]
    train_tgt = np.delete(train_labels, np.s_[index:fold])
    
    
    net3.load_weights('net3.h5')
    history = net3.fit(train_fold, train_tgt, epochs = 25, validation_data = (val_fold, val_tgt))

    histories3.append(history)
    
    
    results = net3.evaluate(test_data, test_labels)
    results_list3.append(results)
    print(results)
    
    index = index + val_split_size
    fold = fold + val_split_size

finish = tm.time()
duration3 = finish - start

0
Train on 54 samples, validate on 18 samples
Epoch 1/25
54/54 [==============================] - 1s 28ms/step - loss: 2.8887 - accuracy: 0.1481 - val_loss: 8.9812 - val_accuracy: 0.1667
Epoch 2/25
54/54 [==============================] - 1s 27ms/step - loss: 6.3321 - accuracy: 0.2222 - val_loss: 9.1852 - val_accuracy: 0.1111
Epoch 3/25
54/54 [==============================] - 1s 26ms/step - loss: 7.0666 - accuracy: 0.1481 - val_loss: 6.0833 - val_accuracy: 0.0000e+00
Epoch 4/25
54/54 [==============================] - 1s 26ms/step - loss: 4.3928 - accuracy: 0.1296 - val_loss: 2.1788 - val_accuracy: 0.1667
Epoch 5/25
54/54 [==============================] - 1s 26ms/step - loss: 1.8589 - accuracy: 0.2778 - val_loss: 1.4172 - val_accuracy: 0.5000
Epoch 6/25
54/54 [==============================] - 1s 27ms/step - loss: 1.7856 - accuracy: 0.4444 - val_loss: 1.5550 - val_accuracy: 0.1667
Epoch 7/25
54/54 [==============================] - 1s 26ms/step - loss: 1.6973 - accuracy: 0.3333 - val

54/54 [==============================] - 1s 26ms/step - loss: 2.0442 - accuracy: 0.2778 - val_loss: 2.2825 - val_accuracy: 0.2778
Epoch 7/25
54/54 [==============================] - 1s 26ms/step - loss: 2.1873 - accuracy: 0.2037 - val_loss: 1.8122 - val_accuracy: 0.3333
Epoch 8/25
54/54 [==============================] - 1s 26ms/step - loss: 1.4531 - accuracy: 0.5185 - val_loss: 2.1981 - val_accuracy: 0.3333
Epoch 9/25
54/54 [==============================] - 1s 26ms/step - loss: 1.4594 - accuracy: 0.5185 - val_loss: 1.6613 - val_accuracy: 0.3889
Epoch 10/25
54/54 [==============================] - 1s 27ms/step - loss: 1.0612 - accuracy: 0.5556 - val_loss: 1.3871 - val_accuracy: 0.3889
Epoch 11/25
54/54 [==============================] - 1s 26ms/step - loss: 0.9654 - accuracy: 0.5741 - val_loss: 1.0478 - val_accuracy: 0.5556
Epoch 12/25
54/54 [==============================] - 1s 27ms/step - loss: 0.7915 - accuracy: 0.7407 - val_loss: 1.1361 - val_accuracy: 0.4444
Epoch 13/25
54/54 [==

In [13]:
avg = np.mean(results_list3, axis=0)
std_dev = np.std(results_list3, axis=0)
print('Average loss and accuracy scores were: ', avg)
print('Standard deviation for loss & accuracy: ', std_dev)
print('Duration in seconds: ', int(duration3))

Average loss and accuracy scores were:  [0.4439797  0.76666668]
Standard deviation for loss & accuracy:  [0.10034725 0.05773502]
Duration in seconds:  145


<h2> Expriment4:</h2>

In [14]:
net4 = models.Sequential()
net4.add(layers.Dense((480), input_shape = (480,6)))
net4.add(layers.Flatten())
net4.add(layers.Dense(240,activation='relu'))
net4.add(layers.Dense(120,activation='relu'))
net4.add(layers.Dense(120,activation='relu'))
net4.add(layers.Dense(7, activation = 'softmax'))
net4.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 480, 480)          3360      
_________________________________________________________________
flatten_4 (Flatten)          (None, 230400)            0         
_________________________________________________________________
dense_17 (Dense)             (None, 240)               55296240  
_________________________________________________________________
dense_18 (Dense)             (None, 120)               28920     
_________________________________________________________________
dense_19 (Dense)             (None, 120)               14520     
_________________________________________________________________
dense_20 (Dense)             (None, 7)                 847       
Total params: 55,343,887
Trainable params: 55,343,887
Non-trainable params: 0
__________________________________________

In [15]:
net4.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
net4.save_weights('net4.h5')  # each split will start with same weights so can directly compare results

histories4 = []
results_list4 = []
index = 0
fold = val_split_size



start = tm.time()

for i in range (k):
    print(i)
    val_fold = train_data[index:fold]
    train_fold = np.delete(train_data, np.s_[index:fold],0)

    val_tgt = train_labels[index:fold]
    train_tgt = np.delete(train_labels, np.s_[index:fold])
    
    
    net4.load_weights('net4.h5')
    history = net4.fit(train_fold, train_tgt, epochs = 25, validation_data = (val_fold, val_tgt))

    histories4.append(history)
    
    
    results = net4.evaluate(test_data, test_labels)
    results_list4.append(results)
    print(results)
    
    index = index + val_split_size
    fold = fold + val_split_size

finish = tm.time()
duration4 = finish - start

0
Train on 54 samples, validate on 18 samples
Epoch 1/25
54/54 [==============================] - 2s 30ms/step - loss: 4.0274 - accuracy: 0.1667 - val_loss: 5.0610 - val_accuracy: 0.1667
Epoch 2/25
54/54 [==============================] - 1s 27ms/step - loss: 6.5335 - accuracy: 0.1296 - val_loss: 6.1429 - val_accuracy: 0.0556
Epoch 3/25
54/54 [==============================] - 1s 27ms/step - loss: 5.5996 - accuracy: 0.1111 - val_loss: 5.3485 - val_accuracy: 0.1111
Epoch 4/25
54/54 [==============================] - 1s 26ms/step - loss: 3.7186 - accuracy: 0.2037 - val_loss: 2.0583 - val_accuracy: 0.2222
Epoch 5/25
54/54 [==============================] - 1s 27ms/step - loss: 1.8932 - accuracy: 0.3148 - val_loss: 1.9089 - val_accuracy: 0.3889
Epoch 6/25
54/54 [==============================] - 1s 27ms/step - loss: 2.0365 - accuracy: 0.3333 - val_loss: 1.8732 - val_accuracy: 0.1111
Epoch 7/25
54/54 [==============================] - 1s 27ms/step - loss: 1.7289 - accuracy: 0.2778 - val_los

54/54 [==============================] - 1s 26ms/step - loss: 1.3912 - accuracy: 0.3519 - val_loss: 1.9244 - val_accuracy: 0.3889
Epoch 7/25
54/54 [==============================] - 1s 27ms/step - loss: 1.5078 - accuracy: 0.3519 - val_loss: 1.6513 - val_accuracy: 0.2778
Epoch 8/25
54/54 [==============================] - 1s 26ms/step - loss: 1.2224 - accuracy: 0.5185 - val_loss: 1.5387 - val_accuracy: 0.4444
Epoch 9/25
54/54 [==============================] - 1s 26ms/step - loss: 1.1761 - accuracy: 0.5556 - val_loss: 1.1171 - val_accuracy: 0.6111
Epoch 10/25
54/54 [==============================] - 1s 26ms/step - loss: 0.9383 - accuracy: 0.7222 - val_loss: 1.3983 - val_accuracy: 0.2778
Epoch 11/25
54/54 [==============================] - 1s 27ms/step - loss: 0.9275 - accuracy: 0.5185 - val_loss: 1.2646 - val_accuracy: 0.3333
Epoch 12/25
54/54 [==============================] - 1s 26ms/step - loss: 0.7601 - accuracy: 0.6481 - val_loss: 0.8583 - val_accuracy: 0.7778
Epoch 13/25
54/54 [==

In [16]:
avg = np.mean(results_list4, axis=0)
std_dev = np.std(results_list4, axis=0)
print('Average loss and accuracy scores were: ', avg)
print('Standard deviation for loss & accuracy: ', std_dev)
print('Duration in seconds: ', int(duration4))

Average loss and accuracy scores were:  [0.54580935 0.75000001]
Standard deviation for loss & accuracy:  [0.06449396 0.05527708]
Duration in seconds:  145


<h2> Expriment5:</h2>

In [17]:
net5 = models.Sequential()
net5.add(layers.Dense((480), input_shape = (480,6)))
net5.add(layers.Flatten())
net5.add(layers.Dense(360,activation='relu'))
net5.add(layers.Dense(360,activation='relu'))
net5.add(layers.Dense(240,activation='relu'))
net5.add(layers.Dense(7, activation = 'softmax'))
net5.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 480, 480)          3360      
_________________________________________________________________
flatten_5 (Flatten)          (None, 230400)            0         
_________________________________________________________________
dense_22 (Dense)             (None, 360)               82944360  
_________________________________________________________________
dense_23 (Dense)             (None, 360)               129960    
_________________________________________________________________
dense_24 (Dense)             (None, 240)               86640     
_________________________________________________________________
dense_25 (Dense)             (None, 7)                 1687      
Total params: 83,166,007
Trainable params: 83,166,007
Non-trainable params: 0
__________________________________________

In [18]:
net5.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
net5.save_weights('net5.h5')  # each split will start with same weights so can directly compare results

histories5 = []
results_list5 = []
index = 0
fold = val_split_size



start = tm.time()

for i in range (k):
    print(i)
    val_fold = train_data[index:fold]
    train_fold = np.delete(train_data, np.s_[index:fold],0)

    val_tgt = train_labels[index:fold]
    train_tgt = np.delete(train_labels, np.s_[index:fold])
    
    
    net5.load_weights('net5.h5')
    history = net5.fit(train_fold, train_tgt, epochs = 25, validation_data = (val_fold, val_tgt))

    histories5.append(history)
    
    
    results = net5.evaluate(test_data, test_labels)
    results_list5.append(results)
    print(results)
    
    index = index + val_split_size
    fold = fold + val_split_size

finish = tm.time()
duration5 = finish - start

0
Train on 54 samples, validate on 18 samples
Epoch 1/25
54/54 [==============================] - 3s 47ms/step - loss: 4.5390 - accuracy: 0.1111 - val_loss: 6.6134 - val_accuracy: 0.2222
Epoch 2/25
54/54 [==============================] - 2s 40ms/step - loss: 9.2058 - accuracy: 0.1667 - val_loss: 10.2480 - val_accuracy: 0.0556
Epoch 3/25
54/54 [==============================] - 2s 40ms/step - loss: 8.7686 - accuracy: 0.1481 - val_loss: 3.5982 - val_accuracy: 0.2778
Epoch 4/25
54/54 [==============================] - 2s 40ms/step - loss: 3.9425 - accuracy: 0.2778 - val_loss: 4.5796 - val_accuracy: 0.3889
Epoch 5/25
54/54 [==============================] - 2s 40ms/step - loss: 3.9363 - accuracy: 0.3519 - val_loss: 2.5044 - val_accuracy: 0.6111
Epoch 6/25
54/54 [==============================] - 2s 39ms/step - loss: 2.1369 - accuracy: 0.4444 - val_loss: 1.7686 - val_accuracy: 0.2222
Epoch 7/25
54/54 [==============================] - 2s 40ms/step - loss: 1.5409 - accuracy: 0.3704 - val_lo

54/54 [==============================] - 2s 39ms/step - loss: 3.2402 - accuracy: 0.2037 - val_loss: 3.5638 - val_accuracy: 0.2778
Epoch 7/25
54/54 [==============================] - 2s 39ms/step - loss: 2.6334 - accuracy: 0.3704 - val_loss: 2.0127 - val_accuracy: 0.4444
Epoch 8/25
54/54 [==============================] - 2s 40ms/step - loss: 1.6022 - accuracy: 0.4074 - val_loss: 1.6528 - val_accuracy: 0.2778
Epoch 9/25
54/54 [==============================] - 2s 40ms/step - loss: 1.4324 - accuracy: 0.3889 - val_loss: 1.3022 - val_accuracy: 0.4444
Epoch 10/25
54/54 [==============================] - 2s 39ms/step - loss: 0.9640 - accuracy: 0.7037 - val_loss: 1.5373 - val_accuracy: 0.3333
Epoch 11/25
54/54 [==============================] - 2s 40ms/step - loss: 1.0302 - accuracy: 0.5926 - val_loss: 1.1438 - val_accuracy: 0.6111
Epoch 12/25
54/54 [==============================] - 2s 40ms/step - loss: 0.7863 - accuracy: 0.6667 - val_loss: 1.1238 - val_accuracy: 0.5556
Epoch 13/25
54/54 [==

In [19]:
avg = np.mean(results_list5, axis=0)
std_dev = np.std(results_list5, axis=0)
print('Average loss and accuracy scores were: ', avg)
print('Standard deviation for loss & accuracy: ', std_dev)
print('Duration in seconds: ', int(duration5))

Average loss and accuracy scores were:  [0.51167821 0.78333335]
Standard deviation for loss & accuracy:  [0.07887788 0.07264831]
Duration in seconds:  218
